In [ ]:
import os
import subprocess
from pathlib import Path

# Adjust these paths as needed
OPENMVG_BIN = "/Applications/OpenHV.app"  # or wherever Homebrew installs openMVG
#OPENMVS_BIN = "/usr/local/bin"  # or custom build path

image_dir = '/Users/richarddvorak/Library/CloudStorage/GoogleDrive-dvor.richard@gmail.com/Můj disk/Škola/Měření/2024/COLMAP/Vaze'
matches_dir = '3D_scan/matches'
reconstruction_dir = '3D_scan/reconstruction'
mvs_dir = '3D_scan/mvs_dir'
output_mesh = "final_mesh.obj"

# Create directories
Path(matches_dir).mkdir(exist_ok=True)
Path(reconstruction_dir).mkdir(exist_ok=True)
Path(mvs_dir).mkdir(exist_ok=True)

# Step 1: Intrinsics analysis
subprocess.run([
    f"{OPENMVG_BIN}/openMVG_main_SfMInit_ImageListing",
    "-i", image_dir,
    "-o", matches_dir,
    "-d", f"{OPENMVG_BIN}/sensor_width_camera_database.txt"
])

# Step 2: Feature extraction
subprocess.run([
    f"{OPENMVG_BIN}/openMVG_main_ComputeFeatures",
    "-i", f"{matches_dir}/sfm_data.json",
    "-o", matches_dir
])

# Step 3: Compute matches
subprocess.run([
    f"{OPENMVG_BIN}/openMVG_main_ComputeMatches",
    "-i", f"{matches_dir}/sfm_data.json",
    "-o", matches_dir
])

# Step 4: Structure from Motion
subprocess.run([
    f"{OPENMVG_BIN}/openMVG_main_IncrementalSfM",
    "-i", f"{matches_dir}/sfm_data.json",
    "-m", matches_dir,
    "-o", reconstruction_dir
])

# Step 5: Convert to OpenMVS format
subprocess.run([
    f"{OPENMVG_BIN}/openMVG_main_openMVG2openMVS",
    "-i", f"{reconstruction_dir}/sfm_data.bin",
    "-d", f"{reconstruction_dir}/undistorted_images",
    "-o", f"{mvs_dir}/scene.mvs"
])

# Step 6: Densify point cloud
subprocess.run([
    f"{OPENMVS_BIN}/DensifyPointCloud",
    f"{mvs_dir}/scene.mvs",
    "-w", mvs_dir
])

# Step 7: Reconstruct mesh
subprocess.run([
    f"{OPENMVS_BIN}/ReconstructMesh",
    f"{mvs_dir}/scene_dense.mvs",
    "-w", mvs_dir
])

# Step 8: Texture mesh
subprocess.run([
    f"{OPENMVS_BIN}/TextureMesh",
    f"{mvs_dir}/scene_dense_mesh.mvs",
    "-w", mvs_dir,
    "-o", output_mesh
])

print(f"✅ Done! Final mesh saved as: {output_mesh}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/richarddvorak/Documents/GitHub/openMVG/src/openMVG/openMVG_main_SfMInit_ImageListing'